In [1]:
import torch
import numpy as np
import torch.nn.functional as F
import torch.nn as A
import pandas as pd
from sklearn.manifold import TSNE
import sklearn.manifold 
from matplotlib import pyplot as plt
import re
import codecs
from numpy  import array
from scipy import stats
# from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
import os
from tensorboardX import SummaryWriter


# PREPROCESSING

In [2]:
corpus_filename = '../data/extracted_emoji_sequences.txt' 

In [3]:
corpus = open(corpus_filename).read().splitlines()

In [4]:
def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens

In [5]:
def onlyEmojiSequences(tokens):
    threshold_emojis = [x for x in tokens if len(x) > 1]
    return threshold_emojis

In [6]:
tokenized_corpus = tokenize_corpus(corpus)
emojiSequences = onlyEmojiSequences(tokenized_corpus)

In [7]:
vocabulary = []
for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)

# this is just the very basic translation both ways plus the length of vocabulary

In [8]:
window_size = 8
idx_pairs = []
# for each sentence
for sentence in tokenized_corpus:
    indices = [word2idx[word] for word in sentence]
    # for each word, threated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array

# sole purpose of this is to have pairs! of target and context word
# super simple once you have figured out the code!

In [9]:
def get_input_layer(word_idx):
    x = torch.zeros(vocabulary_size).device('cuda:0').float()
    x[word_idx] = 1.0
    return x

# this is a one hot encoded something

# TRAINING

In [10]:
testWriter = SummaryWriter()

In [11]:
dtype = torch.float 
device = torch.device("gpu")
dimensionSize = 300
num_epochs = 5
learning_rate = 0.001


firstLayer = torch.randn(dimensionSize, vocabulary_size, device=device, requires_grad=True).float()
secondLayer = torch.randn(vocabulary_size, dimensionSize, device=device, requires_grad=True).float()

for epo in range(num_epochs):
    loss_val = 0
    for data, target in idx_pairs:


        # one hot encoded tensor
        x = get_input_layer(data).float()
        # target word 
        y_true = torch.from_numpy(np.array([target])).device('cuda:0').long()

        # Hidden Layer: gradient magic happening ...
        z1 = torch.matmul(firstLayer, x)
        z2 = torch.matmul(secondLayer, z1)

        # introducing non-linearity
        softmax = A.LogSoftmax(dim=0).device('cuda:0')
        soft_max_output = softmax(z2)

        # neg_log_likelihood
        loss = F.nll_loss(soft_max_output.view(1,-1), y_true)
        loss_val += loss  # this might be to please the  framework, and adding stuff to the gradient calculator
        
        # propagating it back
        loss.backward()
        
        # updating the weights of both layers
        firstLayer.data -= learning_rate * firstLayer.grad.data
        secondLayer.data -= learning_rate * secondLayer.grad.data

        # set the gradients to zero for next iteration
        firstLayer.grad.data.zero_()
        secondLayer.grad.data.zero_()
        
    # this keeps track of the loss, hopefully it does converge
#     if epo % 1000 == 0:    
#         print(f'Loss at epo {epo}: {loss_val/len(idx_pairs)}')

    print(f'Loss at epo {epo}: {loss_val/len(idx_pairs)}')
    testWriter.add_scalar('lossvalue', loss_val, epo)
testWriter.close

Loss at epo 0: 12.061951637268066
Loss at epo 1: 6.53749942779541
Loss at epo 2: 5.4998369216918945
Loss at epo 3: 4.9899773597717285
Loss at epo 4: 4.6804118156433105


<bound method SummaryWriter.close of <tensorboardX.writer.SummaryWriter object at 0x1a23e0a0b8>>

In [8]:
if not os.path.exists("SOFTMAX_trained"):
    os.makedirs("SOFTMAX_trained")

In [12]:
torch.save(secondLayer, os.path.join("SOFTMAX_trained", "newtestHAHA#1.w2v"))

# EVALUATING TRAINED MODEL

In [2]:
loadedLayer = torch.load(os.path.join("SOFTMAX_trained", "1epoch#1.pt"))

In [5]:
# extracts the 508 Annotator Results as the Gold-Standard
corpus_filename = '../data/EmoSim508.json'
corpus = open(corpus_filename).read()
annotator_similarity_score_508 = list(array(re.findall('(?<=_Annotator_Agreement": )(.*?)(?=\})', corpus)))

# glyph_pairs_1016
unicode_pairs_1016 = re.findall('(?<=unicodelong": "\\\)(.*?)(?=")', corpus)    
glyph_pairs_1016 = [codecs.decode(unicode_pairs_1016[x].replace(str('\\\\'),str('\\')).replace('_',''), 'unicode_escape') for x in range(len(unicode_pairs_1016))]

# computation of Cosine Similarity
goldstandard = []
selftrained = []
for x in range(len(annotator_similarity_score_508)):
    cosineSimilarity = None
    
    emoji1 = glyph_pairs_1016.pop(0)
    emoji2 = glyph_pairs_1016.pop(0)
    
    try:
        cosineSimilarity = cosine_similarity(loadedLayer.detach().numpy()[word2idx[emoji1]].reshape(-1,300), loadedLayer.detach().numpy()[word2idx[emoji2]].reshape(-1,300))[0][0]
    except:
        print('the cosine similarity between ' + emoji1 + ' and ' + emoji2 + ' could not be computed.')

    
    if(cosineSimilarity is not None):
        selftrained.append(cosineSimilarity)
        goldstandard.append(annotator_similarity_score_508.pop(0))

# computation of SPEARRANK CORRELATION COEFFICIENT
spearmanRank = stats.spearmanr(goldstandard, selftrained)

print('Der Spearman Rank Correlation Coefficient ist {}'.format(spearmanRank))

the cosine similarity between 🎵 and 🎶 could not be computed.
the cosine similarity between 🎊 and 🎉 could not be computed.
the cosine similarity between ☺ and 😊 could not be computed.
the cosine similarity between ❤ and 💞 could not be computed.
the cosine similarity between 💕 and ❤ could not be computed.
the cosine similarity between 💞 and 💕 could not be computed.
the cosine similarity between 💘 and 💕 could not be computed.
the cosine similarity between 💕 and 😍 could not be computed.
the cosine similarity between 💜 and 💙 could not be computed.
the cosine similarity between 💗 and ❤ could not be computed.
the cosine similarity between 💗 and 💕 could not be computed.
the cosine similarity between ❤ and 💓 could not be computed.
the cosine similarity between 🎤 and 🎶 could not be computed.
the cosine similarity between 💖 and 💕 could not be computed.
the cosine similarity between 💕 and 💓 could not be computed.
the cosine similarity between ❤ and 💙 could not be computed.
the cosine similarity be

In [9]:
word2idx['fds']

NameError: name 'word2idx' is not defined